In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

## Helper functions

In this Jupyter notebook, we will perform data cleaning on the recalls items dataset for all four categories:
- Automotive dataset (7,800 items)
- Food dataset (990 items)
- Consumer Product dataset (1,245 items)
- Medical Product dataset (4,800 items)

In [2]:
def basic_data_cleaning(df):
    
    """
    This function performs a basic data cleaning on the given dataframe.
    
    :param [df]: dataframe of the recalls items
    :type [df]: pd.DataFrame
    
    :return : cleaned dataframe
    :rtype : pd.DataFrame
    """
    
    # drop non pertinent columns
    columns_to_drop = ['Unnamed: 0', 'Starting date:', 'Posting date:', 'Identification number:']
    df = df.drop(columns_to_drop, axis = 1)
    
    # clean up the string entries in 'Item', 'Published by' and 'Audience' columns
    df['Item'] = df['Item'].apply(lambda item: str(item).replace('\n', ''))
    df['Published by'] = df['Published by'].apply(lambda info: str(info).replace('\n', ''))
    df['Audience'] = df['Audience'].apply(lambda info: str(info).replace('\n', ''))
    
    # fill NaN values 
    df['Nb_affected_models'] = df['Nb_affected_models'].fillna(1)
    
    # Convert 'Date' to datetime object and extract Year and Month
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.strftime("%B")
    
    return df

In [3]:
def merge_columns_data(col_version1, col_version2, row):
    
    """
    This function merges two columns that have the same type of informations.
    Note that the https://recalls-rappels.canada.ca website have two different layout versions for the recall items.
    version1 - old version
    version2 - recent version
    
    :param [col_version1]: name of the column feature extracted from the webpage with version 1 layout
    :param [col_version2]: name of the column feature extracted from the webpage with version 2 layout
    :param [row]: row of the dataframe that needs to transform
    :type [col_version1]: string
    :type [col_version2]: string
    :type [row]: series
    
    :return : data of the feature that is now all stored in col_version2
    :rtype: string
    """
    
    if (row[col_version2] == 'nan') or (type(row[col_version2]) != str):
        data = row[col_version1]
    else:
        data = row[col_version2]
    return data

___
## Data Cleaning on Automotive dataset

In [4]:
automotive = pd.read_csv('data/raw_data/rawdata_vehicles.csv')
display(automotive.head(3))
print('Shape:', automotive.shape)
automotive.isna().sum()

,Unnamed: 0.1,Unnamed: 0,Last_updated,Brand,Product,Issue,Nb_affected_models,Alert / recall type,Category,Published by,Audience,Recall type,Identification number
0,0,0,2011-01-12,MILLER,Light Trailer recalled by MILLER ELECTRIC,Vehicle - Suspension,2,Automotive recall,Vehicles - Light Trailer,Transport Canada,General public,automotive recall,2011001
1,1,0,2011-01-18,SPARTAN,Truck - Med. & H.D. recalled by SPARTAN,Vehicle - Suspension,4,Automotive recall,Vehicles - Truck - Med. & H.D.,Transport Canada,General public,automotive recall,2010450
2,2,0,2011-01-19,DUTCHMEN,RV Trailer recalled by DUTCHMEN,Vehicle - Fuel Supply,2,Automotive recall,Vehicles - RV Trailer,Transport Canada,General public,automotive recall,2011009


Shape: (7800, 13)


Unnamed: 0.1              0
Unnamed: 0                0
Last_updated              0
Brand                    30
Product                   0
Issue                     0
Nb_affected_models        0
Alert / recall type       0
Category                  0
Published by              0
Audience                  0
Recall type               0
Identification number     0
dtype: int64

#### Helper functions for cleaning `Vehicles` dataset

In [5]:
def get_car_brand(product):
    """
    This function extracts the brand from the 'Product' column to fill the row with NaN value.
    
    :param [product]: full description of the vehicle with this following format: "[Category] recalled by [Brand]"
    :type [product]: string
    
    :return : brand of the recalled vehicle
    :rtype : string
    
    """
    
    brand = ''
    if 'recalled by' in product:
        brand = product.split('recalled by')[1].lstrip()
    elif 'rappelé par' in product:
        brand = product.split('rappelé par')[1].lstrip()
    return brand


##############################################################
def get_car_category(product):  
    
    """
    This function extracts the category from the 'Product' column
    
    :param [product]: full description of the vehicle with this following format: "[Category] recalled by [Brand]"
    :type [product]: string
    
    :return : category of the recalled vehicle
    :rtype : string
    
    """
    category = ''
    if 'recalled by' in product:
        category = product.split('recalled by')[0].rstrip()
    elif 'rappelé par' in product:
        category = product.split('rappelé par')[0].rstrip()
    return category


##############################################################
def get_car_issue(issue):
    
    """
    This function extracts the issue from the 'Issue' column.
    
    :param [product]: full description of the vehicle issue with this following format: "Vehicle - [Issue]"
    :type [product]: string
    
    :return : issue of the recalled vehicle
    :rtype : string
    
    """
    if ' - ' in issue:
        issue = issue.split(' - ')[-1]
    else:
        issue = automotive.Issue.mode().values[0].split(' - ')[1]
    return issue

#### Data Cleaning - Vehicles dataset

In [6]:
# add new column
automotive['Recall Category'] = 'Vehicles'


# convert 'Date' to datetime object and extract Year, Month
automotive.rename({'Last_updated': 'Date'}, axis=1, inplace=True)
automotive['Date'] = pd.to_datetime(automotive['Date'])
automotive['Year'] = automotive['Date'].dt.year
automotive['Month'] = automotive['Date'].dt.strftime("%B")


# data cleaning
automotive['Audience'] = automotive.Audience.mode().values[0]
automotive['Category'] = automotive['Product'].apply(lambda row: get_car_category(row))
automotive['Issue'] = automotive['Issue'].apply(lambda row: get_car_issue(row))
automotive['Brand'] = automotive.apply(lambda row: get_car_brand(row['Product']) 
                                       if type(row['Brand']) != str else row['Brand'], axis = 1)

# drop redundant and non pertinent columns
cols_to_drop = ['Unnamed: 0.1', 'Unnamed: 0','Alert / recall type', 'Identification number']
automotive.drop(cols_to_drop, axis = 1, inplace = True)

In [7]:
automotive

,Date,Brand,Product,Issue,Nb_affected_models,Category,Published by,Audience,Recall type,Recall Category,Year,Month
0,2011-01-12,MILLER,Light Trailer recalled by MILLER ELECTRIC,Suspension,2,Light Trailer,Transport Canada,General public,automotive recall,Vehicles,2011,January
1,2011-01-18,SPARTAN,Truck - Med. & H.D. recalled by SPARTAN,Suspension,4,Truck - Med. & H.D.,Transport Canada,General public,automotive recall,Vehicles,2011,January
2,2011-01-19,DUTCHMEN,RV Trailer recalled by DUTCHMEN,Fuel Supply,2,RV Trailer,Transport Canada,General public,automotive recall,Vehicles,2011,January
3,2011-01-19,TRIUMPH,Motorcycle recalled by TRIUMPH,Engine,2,Motorcycle,Transport Canada,General public,automotive recall,Vehicles,2011,January
4,2011-01-19,DUCATI,Motorcycle recalled by DUCATI,Engine,1,Motorcycle,Transport Canada,General public,automotive recall,Vehicles,2011,January
...,...,...,...,...,...,...,...,...,...,...,...,...
7795,2023-03-31,JAYCO,Motorhome recalled by JAYCO,Electrical,1,Motorhome,Transport Canada,General public,automotive recall,Vehicles,2023,March
7796,2023-03-31,JAYCO,Motorhome recalled by JAYCO,Suspension,1,Motorhome,Transport Canada,General public,automotive recall,Vehicles,2023,March
7797,2023-03-31,TESLA,SUV recalled by TESLA,Suspension,5,SUV,Vehicles - SUV,General public,automotive recall,Vehicles,2023,March
7798,2023-03-31,FOREST RIVER,RV Trailer recalled by FOREST RIVER,Structure,1,RV Trailer,Transport Canada,General public,automotive recall,Vehicles,2023,March


In [8]:
# The automotive dataset is cleaned.
automotive.isna().sum()

Date                  0
Brand                 0
Product               0
Issue                 0
Nb_affected_models    0
Category              0
Published by          0
Audience              0
Recall type           0
Recall Category       0
Year                  0
Month                 0
dtype: int64

___
## Data Cleaning on Food dataset

In [9]:
food = pd.read_csv('data/raw_data/rawdata_food.csv')
display(food.head(3))
print('Shape:', food.shape)
food.isna().sum()

,Unnamed: 0,Date,Item,Brand,Product,Issue,Nb_affected_models,Alert / recall type,Category,Companies,...,Hazard classification:,Source of recall:,Recalling firm:,Distribution:,Extent of the product distribution:,CFIA reference number:,Issue:,Audience:,Identification number:,Posting date:
0,0,2023-03-30,\nKomera Original brand Seasoned Hot Pepper Sa...,Komera Original,Seasoned Hot Pepper Sauce,Food - Microbial Contamination - Clostridium b...,2.0,Food recall warning,Food - Condiments,Komera Inc.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2023-03-29,\nSolstice Ciderworks brand Haskap Cider recal...,Solstice Ciderworks,Haskap Cider,Food - Microbial Contamination - Non harmful (...,2.0,Notification,Food - Beverages - Alcoholic,Solstice Ciderworks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2023-03-29,\nVarious brands of Chicken Soup Products reca...,no name,Chicken Soup Products,Food - Microbial Contamination - Other,5.0,Notification,Food - Multiple food items,Les Aliments BCI Inc.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape: (990, 29)


Unnamed: 0                               0
Date                                     0
Item                                     0
Brand                                  682
Product                                654
Issue                                  654
Nb_affected_models                     654
Alert / recall type                    654
Category                               654
Companies                              658
Published by                           654
Audience                               655
Recall class                           667
Recall type                              0
Identification number                  845
Starting date:                         336
Type of communication:                 336
Alert sub-type:                        346
Subcategory:                           338
Hazard classification:                 346
Source of recall:                      336
Recalling firm:                        351
Distribution:                          351
Extent of t

#### Helper functions for cleaning `Food` dataset

In [10]:
def get_food_category(category):

    """
    This function extracts the category from the 'Category' column
    
    :param [product]: full description of the food item with this following format: "Food - [category]"
    :type [product]: string
    
    :return : category of the recalled food item
    :rtype : string
    
    """
    
    if category is not np.nan:
        category = category.split(' - ')[-1]
    else:
        category = 'Not Categorized'
    return category


##############################################################
def get_recall_class(row):
    
    """
    This function corrects the data in the 'Recall class' column.
    
    :param [product]: recall class
    :type [product]: string
    
    :return : recall class
    :rtype : string
    
    """
    
    if 'Class' in str(row['Recall class']):
        recall_class = row['Recall class']

    else:
        recall_class = food['Recall class'].mode()[0] 
    return recall_class


##############################################################
def get_food_mainIssue(issue):
    
    """
    This function helps to reduce repetitive/group data in the 'Issue' column.
    
    :param [product]: full description of the food issue
    :type [product]: string
    
    :return : issue of the recalled food item
    :rtype : string
    
    """
    
    issue = str(issue)
    
    if re.search(r"\bAllergen\b|\bGluten\b", issue):
        issue = 'Allergen'
    elif re.search(r"\bMicrobiol\b|\bMicrobial\b|\bMicrobiological\b", issue):
        issue = 'Microbiological Contamination'
    elif 'Extraneous Material' in issue:
        issue = 'Extraneous Material'
    else:
        issue = 'Other'
    return issue


##############################################################
def clean_food_audience_col(audience):
    """
    This function helps to reduce redundant data in the 'Audience' column by grouping similar informations together.
    
    :param [product]: full description of the audience
    :type [product]: string
    
    :return : audience
    :rtype : string
    
    """
    
    if audience is not np.nan:
        audience = audience.title()
        
        if re.search(r"\bRetail\b|\bWarehouse\b|\bManufacturers\b", audience):
            audience = "Retail/Warehouse/Manufacturers"
            
        elif re.search(r"\bHotels\b|\bHotel\b", audience):
            audience = "Hotel/Restaurant/Institutional"

        else:
            audience = "General Public"
    else:
        audience = "General Public"
    return audience

#### Data Cleaning - Food Dataset

In [11]:
food = basic_data_cleaning(food)
food['Recall Category'] = 'Food'


# merge columns with same informations
food['Recall class'] = food.apply(lambda row: merge_columns_data('Hazard classification:', 'Recall class',row), axis = 1) 
food['Published by'] = food.apply(lambda row: merge_columns_data('Source of recall:', 'Published by',row), axis = 1) 
food['Audience'] = food.apply(lambda row: merge_columns_data('Extent of the product distribution:', 'Audience',row), axis = 1)
food['Audience'] = food['Audience'].apply(lambda audience: clean_food_audience_col(audience))


# extract data from the columns
food['Sub Issue'] = food.apply(lambda row: row['Issue'].split(' - ')[1] if type(row['Issue']) == str else row['Subcategory:'], axis = 1)
food['Category'] = food['Category'].apply(lambda row: get_food_category(row))
food['Recall class'] = food.apply(lambda row: get_recall_class(row), axis = 1)
food['Food_Issue'] = food['Sub Issue'].apply(lambda row: get_food_mainIssue(row))


# drop redundant and non pertinent columns
food.drop(['Recalling firm:', 'Hazard classification:', 'Subcategory:', 'Distribution:', 'Identification number','Alert / recall type',
           'Issue','Issue:', 'Audience:', 'Companies','Extent of the product distribution:', 'Type of communication:', 
           'Alert sub-type:', 'Source of recall:', 'CFIA reference number:', 'Brand', 'Product'], axis = 1, inplace = True)

In [12]:
food

,Date,Item,Nb_affected_models,Category,Published by,Audience,Recall class,Recall type,Year,Month,Recall Category,Sub Issue,Food_Issue
0,2023-03-30,Komera Original brand Seasoned Hot Pepper Sauc...,2.0,Condiments,Canadian Food Inspection Agency,General Public,Class 1,food recall warning,2023,March,Food,Microbial Contamination,Microbiological Contamination
1,2023-03-29,Solstice Ciderworks brand Haskap Cider recalle...,2.0,Alcoholic,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 1,notification,2023,March,Food,Microbial Contamination,Microbiological Contamination
2,2023-03-29,Various brands of Chicken Soup Products recall...,5.0,Multiple food items,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 1,notification,2023,March,Food,Microbial Contamination,Microbiological Contamination
3,2023-03-28,Certain Wonton King brand Wontons recalled due...,5.0,Multiple food items,Canadian Food Inspection Agency,General Public,Class 1,food recall warning,2023,March,Food,Allergen,Allergen
4,2023-03-24,Green Sun brand Dark Chocolate Covered Pineapp...,2.0,"Candy, confectionary, snacks and sweeteners",Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 1,notification,2023,March,Food,Allergen,Allergen
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2012-08-10,Certain Leadbetters brand Lean Ground Beef T...,1.0,Not Categorized,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 2,food recall warning,2012,August,Food,Extraneous Material,Extraneous Material
986,2012-08-10,Undeclared peanut in certain Organic Select ...,1.0,Not Categorized,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 3,food recall warning,2012,August,Food,Allergen - Peanut,Allergen
987,2012-08-10,Undeclared peanut in various Organic Pine Nuts,1.0,Not Categorized,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 3,food recall warning,2012,August,Food,Allergen - Peanut,Allergen
988,2012-08-08,Various milk may contain extraneous material,1.0,Not Categorized,Canadian Food Inspection Agency,Retail/Warehouse/Manufacturers,Class 3,food recall warning,2012,August,Food,Extraneous Material,Extraneous Material


In [13]:
# The food dataset is cleaned.
food.isna().sum()

Date                  0
Item                  0
Nb_affected_models    0
Category              0
Published by          0
Audience              0
Recall class          0
Recall type           0
Year                  0
Month                 0
Recall Category       0
Sub Issue             2
Food_Issue            0
dtype: int64

___
## Data Cleaning on Consumer product dataset

In [14]:
consumer = pd.read_csv('data/raw_data/rawdata_consumer.csv')
display(consumer.head(3))
print('Shape:', consumer.shape)
consumer.isna().sum()

,Unnamed: 0.1,Unnamed: 0,Date,Item,Brand,Product,Issue,Nb_affected_models,Alert / recall type,Category,...,Audience,Starting date:,Posting date:,Type of communication:,Subcategory:,Source of recall:,Issue:,Audience:,Identification number:,Identification number
0,0,0,2023-03-31,\nThe Laundress fabric conditioners recalled d...,NaN,The Laundress fabric conditioners.,Consumer products - Chemical hazard,5.0,Consumer product recall,Consumer product - Other,...,General public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,2023-03-31,\nHealth Canada warns that YANRU Baby Walkers ...,NaN,YANRU Baby Walkers,Consumer products - Injury hazard,12.0,Consumer product advisory,Consumer product - Household items,...,General public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2023-03-30,\nBattery Packs within Higdon Outdoors XS Seri...,NaN,Battery Packs within Higdon Outdoors XS Series...,Consumer products - Fire hazard,18.0,Consumer product recall,Consumer product - Outdoor living,...,General public,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape: (1245, 23)


Unnamed: 0.1                 0
Unnamed: 0                   0
Date                         0
Item                         0
Brand                     1156
Product                    881
Issue                      881
Nb_affected_models         881
Alert / recall type        881
Category                   881
Companies                  891
Published by               881
Recall type                  0
Audience                   882
Starting date:             364
Posting date:              394
Type of communication:     364
Subcategory:               364
Source of recall:          364
Issue:                     378
Audience:                  379
Identification number:     364
Identification number     1244
dtype: int64

#### Helper functions for cleaning `Consumer Product` dataset

In [15]:
def merge_IssueOrCategory_cols(col_version1, col_version2, row):
    
    """
    This function merges two columns that have the same type of informations.
    The columns are either 'Issue:' with 'Issue' or 'Subcategory:' with 'Category'.
    
    Note that the https://recalls-rappels.canada.ca website have two different versions for the recall items.
    version1 - old version
    version2 - recent version
    
    :param [col_version1]: name of the feature extracted from the webpage with version 1 layout ('Issue:' or 'Subcategory:')
    :param [col_version2]: name of the feature extracted from the webpage with version 2 layout ('Issue' or 'Category')
    :param [row]: row of the dataframe that needs to transform
    :type [col_version1]: string
    :type [col_version2]: string
    :type [row]: series
    
    :return : information of issue/category of the recalled product
    :rtype: string
    """
    
    issue_category = ''
    data = row[col_version2]
    if data is not np.nan:
        if ' - ' in data:
            issue_category = data.split(' - ')[1]
        else:
            issue_category = data
    else:
        issue_category = row[col_version1]
        
    if issue_category is not np.nan:
        issue_category = issue_category.split(',')[0].title()
    else:
        issue_category = 'Other'

    return issue_category


##############################################################
def clean_consumer_category(category):
    
    """
    This function helps to reduce redundant data in the 'Category' column by grouping similar informations together.
    
    :param [product]: full description of the product category
    :type [product]: string
    
    :return : category of the recalled item
    :rtype : string
    
    """
    category = category.title()
    if re.search(r"\bToys\b|\bGames\b", category):
        category = "Toys And Games"
        
    elif 'Sports' in category:
        category = "Sports And Fitness"
        
    elif 'Hobby' in category:
        category = "Hobby And Craft Items"  
        
    elif 'Children' in category:
        category = "Children's Products"  
        
    elif re.search(r"\bBeauty And Personal Care\b|\bCosmetics\b", category):
        category = "Beauty And Personal Care"
        
    elif re.search(r"\bCannabis\b|\bVaping\b|\bDrugs\b",category):
        category = "Cannabis/Vaping/Tobacco products"
        
    elif re.search(r"\bConsumer Product\b|\bOther\b|\bMiscellaneous\b|\bMedical Device\b", category):
        category = "Other"
    
    return category


##############################################################
def clean_issue_cols(issue):
        
    """
    This function helps to reduce redundant data in the 'Issue' column by grouping similar informations together.
    
    :param [product]: full description of the product issue
    :type [product]: string
    
    :return : issue of the recalled item
    :rtype : string
    
    """
    
    issue = issue.title()
    
    if re.search(r"\bQuality\b|\bDosage\b", issue):
        issue = "Product Quality"
        
    elif re.search(r"\bLack Of Efficacy\b|\bPerformance\b", issue):
        issue = "Performance Issue"
        
    elif re.search(r"\bLabel\b|\bLabelling\b|\bPackaging\b", issue):
        issue = "Labelling And Packaging"
        
    elif "Information" in issue:
        issue = "Important Safety Information"
        
    elif re.search(r"\bUnauthorized\b|\bUnauthorised\b", issue):
        issue = "Unauthorized Products/Device"
        
    elif re.search(r"\bSupply\b|\bWithdrawal\b", issue):
        issue = "Supply/Product Withdrawal"
    
    elif "Adverse" in issue:
        issue = "Increased Risk Or New Adverse Events"
        
    elif "Contamination" in issue:
        issue = "Microbial Contamination/Contamination"
        
    elif re.search(r"\bDevice Compatibility\b|\bDefect\b", issue):
        issue = "Software/Mechanical/Electrical/Manufacturing Defect"
        
    elif re.search(r"\bUsage\b|\bCovid\b|\bFood\b|\bConsumer\b|\bHealth\b|\bOther\b", issue):
        issue = "Other"  

    return issue


##############################################################
def clean_consumer_audience_col(audience):
    """
    This function helps to reduce redundant data in the 'Audience' column by grouping similar informations together.
    
    :param [product]: full description of the audience
    :type [product]: string
    
    :return : audience
    :rtype : string
    
    """
    if audience is not np.nan:
        audience = audience.title()
        if re.search(r"\bHealth\b|\bHealthcare\b|\bHospitals\b", audience):
            audience = "Healthcare Professionals"
        else:
            audience = "General Public"
    else:
        audience = "General Public"
    return audience

#### Data Cleaning - Consumer Product

In [16]:
consumer = basic_data_cleaning(consumer)
consumer['Recall Category'] = 'Consumer product'


# merge columns with similar type of informations and performs data cleaning
consumer['Audience'] = consumer.apply(lambda row: merge_columns_data('Audience:', 'Audience', row), axis =1)
consumer['Audience'] = consumer['Audience'].apply(lambda audience: clean_consumer_audience_col(audience))
consumer['Published by'] = consumer.apply(lambda row: merge_columns_data('Source of recall:', 'Published by', row), axis =1)


# merge columns 'Issue' with 'Issue:' and 'Subcategoy:' with 'Category' and extract data
consumer['Issue'] = consumer.apply(lambda row:  merge_IssueOrCategory_cols('Issue:', 'Issue', row), axis = 1)
consumer['Issue'] = consumer['Issue'].apply(lambda row: clean_issue_cols(row))
consumer['Category'] = consumer.apply(lambda row:  merge_IssueOrCategory_cols('Subcategory:', 'Category', row), axis = 1)
consumer['Category'] = consumer['Category'].apply(lambda category: clean_consumer_category(category))


# drop redundant and non pertinent columns
consumer.drop(['Unnamed: 0.1', 'Brand', 'Product', 'Source of recall:', 'Issue:', 'Subcategory:', 'Type of communication:',
               'Audience:','Companies','Identification number', 'Alert / recall type'], axis = 1, inplace = True)

In [17]:
consumer

,Date,Item,Issue,Nb_affected_models,Category,Published by,Recall type,Audience,Year,Month,Recall Category
0,2023-03-31,The Laundress fabric conditioners recalled due...,Chemical Hazard,5.0,Other,Health Canada,consumer product recall,General Public,2023,March,Consumer product
1,2023-03-31,Health Canada warns that YANRU Baby Walkers ma...,Injury Hazard,12.0,Household Items,Health Canada,consumer product advisory,General Public,2023,March,Consumer product
2,2023-03-30,Battery Packs within Higdon Outdoors XS Series...,Fire Hazard,18.0,Outdoor Living,Health Canada,consumer product recall,General Public,2023,March,Consumer product
3,2023-03-30,Pure Sunfarms Corp. recalls two lots of Origin...,Labelling And Packaging,3.0,Cannabis/Vaping/Tobacco products,Health Canada,consumer product recall,General Public,2023,March,Consumer product
4,2023-03-29,Bamboo Nursing Hoodies recalled due to Choking...,Choking Hazard,5.0,Clothing And Accessories,Health Canada,consumer product recall,General Public,2023,March,Consumer product
...,...,...,...,...,...,...,...,...,...,...,...
1240,2018-01-19,Quality Goods I.M.D. Inc. recalls various ch...,Product Safety,1.0,Children's Products,Health Canada,consumer product recall,General Public,2018,January,Consumer product
1241,2018-01-18,Panasonic Canada recalls Panasonic LCD Hospi...,Product Safety,1.0,Household Items,Health Canada,consumer product recall,General Public,2018,January,Consumer product
1242,2018-01-18,Bed Bath & Beyond Canada L.P. recalls Hudson...,Other,1.0,Household Items,Health Canada,consumer product recall,General Public,2018,January,Consumer product
1243,2018-01-16,"FUJIFILM Canada,Inc. recalls AC-5VF Power Ad...",Electrical Hazard,1.0,Electronics,Health Canada,consumer product recall,General Public,2018,January,Consumer product


In [18]:
# The consumer dataset is cleaned.
consumer.isna().sum()

Date                  0
Item                  0
Issue                 0
Nb_affected_models    0
Category              0
Published by          0
Recall type           0
Audience              0
Year                  0
Month                 0
Recall Category       0
dtype: int64

___
## Data Cleaning on Medical product dataset

In [19]:
medical = pd.read_csv('data/raw_data/rawdata_medical.csv')
display(medical.head(3))
print('Shape:', medical.shape)
medical.isna().sum()

,Unnamed: 0.1,Unnamed: 0,Date,Item,Brand,Product,Issue,Nb_affected_models,Alert / recall type,Category,...,Recall class,Starting date:,Posting date:,Type of communication:,Subcategory:,Source of recall:,Issue:,Audience:,Identification number:,Hazard classification:
0,0,0,2023-03-31,\nMridium MRI Infusion System - Syringe Adapte...,Iradimed Corporation,Mridium MRI Infusion System - Syringe Adapter Set,Medical devices - Performance issue,2.0,Health product recall,Health product - Medical device - General hosp...,...,Type II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,2023-03-31,\nMagnesium Oxide 420 mg: The dissolution is o...,NaN,Magnesium Oxide 420 mg,Health products - Product quality,2.0,Health product recall,Health product - Natural health products,...,Type II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2023-03-31,\nPhilips Azurion System\n,PHILIPS MEDICAL SYSTEMS NEDERLAND B.V.,Philips Azurion System,Medical devices - Performance issue,5.0,Health product recall,Health product - Medical device - Radiology,...,Type II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape: (4184, 24)


Unnamed: 0.1                 0
Unnamed: 0                   0
Date                         0
Item                         0
Brand                     3544
Product                   3368
Issue                     3368
Nb_affected_models        3368
Alert / recall type       3368
Category                  3368
Companies                 3462
Published by              3368
Audience                  3373
Recall type                  0
Recall class              3455
Starting date:             816
Posting date:             1517
Type of communication:     816
Subcategory:               820
Source of recall:          816
Issue:                     864
Audience:                  860
Identification number:     816
Hazard classification:    1147
dtype: int64

#### Helper functions for cleaning `Medical Product` dataset

In [20]:
def clean_medical_category(category):
    
    """
    This function helps to reduce repetitive data in the 'Category' column by grouping similar informations together.
    
    :param [product]: full description of the medical product category
    :type [product]: string
    
    :return : category of the recalled medical item
    :rtype : string
    
    """
    category = category.title()
    
    if re.search(r"\bChildren\b|\bInfant\b", category):
        category = "Infant/Children's Products"
        
    elif "Health Product" in category:
        category = "Natural Health Products"
        
    elif "Biologic" in category:
        category = "Biologic Or Vaccine"
    
    elif "Veterinary" in category:
        category = "Veterinary Drugs"
    
    elif re.search(r"\bRadiopharmaceuticals\b|\bDrugs\b", category):
        category = "Drugs"
        
    elif re.search(r"\bCosmetics\b|\bOther\b|\bMiscellaneous\b", category):
        category = "Other"
        
    return category


##############################################################
def clean_medical_audience_col(audience):
    """
    This function helps to reduce redundant data in the 'Audience' column by grouping similar informations together.
    
    :param [product]: full description of the audience
    :type [product]: string
    
    :return : audience
    :rtype : string
    
    """
        
    if audience is not np.nan:
        audience = audience.title()
        
        if re.search(r"\bHealth\b|\bHealthcare\b|\bHospitals\b", audience):
            audience = "Healthcare Professionals"
            
        elif re.search(r"\bRetail\b|\bIndustry\b|\bWarehouse\b", audience):
            audience = "Industry/Retail/Warehouse"

        else:
            audience = "General Public"
    else:
        audience = "General Public"
    return audience

#### Data Cleaning - Medical dataset

In [21]:
medical = basic_data_cleaning(medical)
medical['Recall Category'] = 'Health product'


# merge columns with similar type of informations
medical['Audience'] = medical.apply(lambda row: merge_columns_data('Audience:', 'Audience', row), axis =1)
medical['Audience'] = medical['Audience'].apply(lambda audience: clean_medical_audience_col(audience))
medical['Recall class'] = medical.apply(lambda row: merge_columns_data('Hazard classification:', 'Recall class', row), axis =1)
medical['Published by'] = medical.apply(lambda row: merge_columns_data('Source of recall:', 'Published by', row), axis =1)


# merge columns 'Issue' with 'Issue:' and 'Subcategoy:' with 'Category' and extract/clean data
medical['Issue'] = medical.apply(lambda row:  merge_IssueOrCategory_cols('Issue:', 'Issue', row), axis = 1)
medical['Issue'] = medical['Issue'].apply(lambda row: clean_issue_cols(row))
medical['Category'] = medical.apply(lambda row:  merge_IssueOrCategory_cols('Subcategory:', 'Category', row), axis = 1)
medical['Category'] = medical['Category'].apply(lambda category: clean_medical_category(category))


# drop redundant and non pertinent columns
medical.drop(['Unnamed: 0.1', 'Source of recall:', 'Type of communication:','Audience:', 'Subcategory:', 'Companies',
              'Hazard classification:', 'Issue:', 'Alert / recall type', 'Brand','Product'], axis = 1, inplace = True)

In [22]:
medical

,Date,Item,Issue,Nb_affected_models,Category,Published by,Audience,Recall type,Recall class,Year,Month,Recall Category
0,2023-03-31,Mridium MRI Infusion System - Syringe Adapter Set,Performance Issue,2.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2023,March,Health product
1,2023-03-31,Magnesium Oxide 420 mg: The dissolution is out...,Product Quality,2.0,Natural Health Products,Health Canada,General Public,health product recall,Type II,2023,March,Health product
2,2023-03-31,Philips Azurion System,Performance Issue,5.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2023,March,Health product
3,2023-03-30,Zenith Branch Endovascular Graft Iliac Bifurca...,Performance Issue,2.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2023,March,Health product
4,2023-03-30,Trifecta Heart Valve,Performance Issue,3.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2023,March,Health product
...,...,...,...,...,...,...,...,...,...,...,...,...
4179,2018-01-08,"IGEA Fast - Multistandard System, Probe, Pro...",Medical Devices,1.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2018,January,Health product
4180,2018-01-08,Confidence and Confidence Plus Kit Spinal Ce...,Medical Devices,1.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2018,January,Health product
4181,2018-01-08,Grace Hysterectomy Pack and Ensemble Prothes...,Medical Devices,1.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2018,January,Health product
4182,2018-01-08,Non-Filtered Curlin IV Administration Sets w...,Medical Devices,1.0,Medical Device,Health Canada,Healthcare Professionals,health product recall,Type II,2018,January,Health product


In [23]:
# The medical dataset is cleaned enough. 
# Notice that the 'Recall class' column still contains NaN values. However, some medical items do not provide the recall class.
medical.isna().sum()

Date                    0
Item                    0
Issue                   0
Nb_affected_models      0
Category                0
Published by            0
Audience                0
Recall type             0
Recall class          418
Year                    0
Month                   0
Recall Category         0
dtype: int64

___
## Save cleaned datasets to csv files

In [24]:
# concatenate all categories into one new dataset
all_items = pd.concat([automotive, food, consumer, medical], ignore_index = True, axis = 0)
all_items = all_items[['Date', 'Year', 'Month', 'Nb_affected_models', 'Audience', 'Recall Category']]

```python
# save cleaned datasets to csv files
all_items.to_csv("all_items_cleaned.csv")
automotive.to_csv("automotive_cleaned.csv")
food.to_csv("food_cleaned.csv")
consumer.to_csv("consumer_cleaned.csv")
medical.to_csv("medical_cleaned.csv")
```